# Representations of Color
Converting input Hex codes to RGB and HSL.

In [1]:
# Assuming input six values
entries = input("Enter list of colors (#FF0000, #00ff00, #fff):").split()
entries

['#FF0000,', '#00ff00,', '#fff']

In [ ]:
def hexFormat(s:str):

    s = s.replace('#','')
    
    if len(s) == 3:
        s = [s[i] * 2 for i in range(3)] 

    return ''.join(s)

In [ ]:
colors = [hexFormat(s) for s in entries]
colors

## Hex to RGB
Convert every set of two hexidecimals to decimals.

In [ ]:
def hexToRGB(h:str):
    # hexadecimal to decimal
    return [int(h[i:i+2], 16) for i in (0, 2, 4)]

In [13]:
rgb_colors = [hexToRGB(i) for i in colors]
rgb_colors

[[255, 0, 0], [0, 255, 0], [255, 255, 255]]

## RGB to HSL
This human-friendly conversion allows us to represent color on a color wheel by determining where the color point exists in relation to pure red, green, or blue. We can do this in 4 steps:

**Step 1**. Calculate the max, min, and chroma.  

$$
\begin{alignat*}{3}
M = max(R, G, B) \\
m = min(R, G, B) \\
C = range(R, G, B) = M - m
\end{alignat*}
$$

**Step 2**. Calculate the hue measured in degrees `[0°, 360°)`.

$$
H' = 
\begin{cases}
\text{undefined}, & \text{if } C = 0 \\
\frac{G-B}{C} \bmod 6, & \text{if } M = R \\
\frac{B-R}{C} + 2, & \text{if } M = G \\
\frac{R-G}{C} + 4, & \text{if } M = B
\end{cases}
$$

Then, $H = 60° × H'$

**Step 3**. Calculate Lightness measuring color intensity using set `[0%, 100%)`.

$$
L = mid(R, G, B) = \frac{1}{2}(M + m) 
$$

**Step 4**. Calculate Saturation determined by the amount of grayscale or pure color from `[0%, 100%)`.

$$
S_L =
\begin{cases}
0, & \text{if } L = 1 \text{ or } L = 0 \\
\frac{C}{1-|2L-1|}, & \text{otherwise}
\end{cases}
$$


See [wiki page](https://en.wikipedia.org/wiki/HSL_and_HSV).

In [14]:
def rgbToHSL(rgb:list):
    # Change range from 0-255 to 0-100%
    r = rgb[0] / 255
    g = rgb[1] / 255
    b = rgb[2] / 255

    # Determine Chroma component values 
    cmax = max(r, g, b)
    cmin = min(r, g, b)
    delta = cmax - cmin

    # Calculate Hue: Distance from white (0-350 degrees)
    if delta == 0:
        h = 0
    elif cmax == r:                # more red
        h = (g - b) / delta % 6
    elif cmax == g:
        h = (b - r) / delta + 2.0   # more green
    else:                       # more blue
        h = (r - g) / delta + 4.0

    # Calulate Lightness: Midrange of RGB (0-100%)
    l = (cmax + cmin) / 2

    # Calulate Saturation: Gray to Pure (0-100%)
    if delta == 0:
        s = 0
    else:
        s = delta / (1 - abs(2 * l - 1))
    

    h = round(h * 60)
    s = round(s, 2)
    l = round(l, 2)
    return [h, s, l]

In [16]:
hsl_colors = [rgbToHSL(i) for i in rgb_colors]
hsl_colors

[[0, 1.0, 0.5], [120, 1.0, 0.5], [0, 0, 1.0]]

## Lightness Adjustments
Some recommended lightness adjustment values:

- Subtle: ±10%
- Moderate: ±20%
- Dramatic: ±30%
- High contrast: ±40%

Generated by *ClaudeAI*

In [33]:
def adjustLightness(hsl:list, amount=20, lighter=True):
    h = hsl[0]
    s = hsl[1]
    l = hsl[2]

    if lighter: 
        return [h, s, min(100, l + amount)]
    else: 
        return [h, s, max(0, l - amount)]

In [42]:
light_colors = [adjustLightness(hsl_colors[i], amount = 0.2, lighter=True) for i in range(len(hsl_colors))]
dark_colors = [adjustLightness(hsl_colors[i], amount = 0.2, lighter=False) for i in range(len(hsl_colors))]

In [45]:
for i in range(len(hsl_colors)):
    print("Light:", light_colors[i])
    print("Base:", hsl_colors[i])
    print("Dark:", dark_colors[i])
    print('---')

Light: [0, 1.0, 0.7]
Base: [0, 1.0, 0.5]
Dark: [0, 1.0, 0.3]
---
Light: [120, 1.0, 0.7]
Base: [120, 1.0, 0.5]
Dark: [120, 1.0, 0.3]
---
Light: [0, 0, 1.2]
Base: [0, 0, 1.0]
Dark: [0, 0, 0.8]
---


# HSL to Hex
Returning to Tableau compatible colors using built-in hex function.

### HSL to RGB


In [ ]:
# Find chroma
chroma = (1 - abs(2*l - 1)) * s

# Find point (r,g,b) of RGB cube
X = chroma * (1 - abs((h/60) % 2 - 1))

if      0 <= h < 1: rgb = [chroma, X, 0]
elif    1 <= h < 2: rgb = [X, chroma, 0]
elif    2 <= h < 3: rgb = [0, chroma, X]
elif    3 <= h < 4: rgb = [0, X, chroma]
elif    4 <= h < 5: rgb = [X, 0, chroma]
elif    5 <= h < 6: rgb = [chroma, 0, X]

# Match the lightness
m = l - chroma/2
RGB = [int((i+m) * 255) for i in rgb]
RGB

### RGB to Hex

In [ ]:
# step 3: decimal RGB values in range 0-255
RGB = [round(i * 255) for i in [r,g,b]]

In [ ]:
# RGB values to hexadecimals
hexCode = [str(hex(int(i))[2:]) for i in RGB]
hexCode

In [ ]:
# six digit format
for i in range(len(hexCode)):
    if len(hexCode[i]) != 2:
        hexCode[i] *= 2 

hexCode = ''.join(hexCode)

In [ ]:
hexCode == colorCode

# Next steps

Adam's flow:

1. input name, palette type and codes
2. tokenizes colors into multiple rows
3. checks that entries are hex codes
4. Group by name
5. handles 3 and 6 hex formats
6. format colors (<color>#123456</color>)
7. Format palettes by replacing $ with newlines 
8. Export code


Mods:

1. +Check box for palette types
2. ok
3. ok
5. move up
4. Split flow by name and type
    - [ ] Categorical: **group** base colors
    - [ ] Sequential: **adjust lightness** for each base color
    - [ ] Diverging: **generate complementary color** for each base color
    - [ ] Transparent: **output row** with #FFFFFF00
6. ok
7. +tabs
- [ ] Format headers
8. ok

